In [ ]:
import numpy as np
np.random.seed(1337)
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense,LSTM,TimeDistributed
from keras.optimizers import Adam

In [ ]:
BATCH_START = 0
TIME_STEPS = 20
BATCH_SIZE = 50
INPUT_SIZE = 1
OUTPUT_SIZE = 1
CELL_SIZE = 20
LR = 0.006

In [3]:
def get_batch():
    global BATCH_START, TIME_STEPS
    # xs shape (1000,) -- (50,20)
    xs = np.arange(BATCH_START, BATCH_START+TIME_STEPS*BATCH_SIZE).reshape((BATCH_SIZE, TIME_STEPS)) / (10*np.pi)
    seq = np.sin(xs)
    res = np.cos(xs)
    BATCH_START += TIME_STEPS
    # plt.plot(xs[0, :], res[0, :], 'r', xs[0, :], seq[0, :], 'b--')
    # plt.show()
    return [seq[:, :, np.newaxis], res[:, :, np.newaxis], xs]

In [4]:
#建立序列模型
model = Sequential()

#建立LSTM循环层
#return_sequential代表是否每一个循环都输出值
#stateful代表batch与batch之间是否有关系，我们这里不设置
model.add(
    LSTM(units=20,input_shape=(20,1),return_sequences=True)
)

#添加输出层
#TimeDistributed可以使循环层中的每一个序列的输出值都经过一个Dense进行输出
model.add(TimeDistributed(Dense(1)))

#设置损失函数，优化方法
adam = Adam(LR)
model.compile(
    optimizer = adam,
    loss = 'mse'
)

In [5]:
#数据准备
X_batch, Y_batch, xs = get_batch()
print(X_batch.shape)
print(Y_batch.shape)
print(xs.shape)

(50, 20, 1)
(50, 20, 1)
(50, 20)


In [6]:
#模型训练 方式1
cost = model.train_on_batch(X_batch, Y_batch)
cost

0.5107169

In [7]:
#模型训练 方式2
model.fit(X_batch,Y_batch,batch_size=20,epochs=10)

Epoch 1/10
50/50 [==============================] - 0s 2ms/step - loss: 0.5042
Epoch 2/10
50/50 [==============================] - 0s 2ms/step - loss: 0.4974
Epoch 3/10
50/50 [==============================] - 0s 2ms/step - loss: 0.4844
Epoch 4/10
50/50 [==============================] - 0s 2ms/step - loss: 0.4830
Epoch 5/10
50/50 [==============================] - 0s 2ms/step - loss: 0.4760
Epoch 6/10
50/50 [==============================] - 0s 3ms/step - loss: 0.4625
Epoch 7/10
50/50 [==============================] - 0s 2ms/step - loss: 0.4545
Epoch 8/10
50/50 [==============================] - 0s 2ms/step - loss: 0.4496
Epoch 9/10
50/50 [==============================] - 0s 1ms/step - loss: 0.4488
Epoch 10/10
50/50 [==============================] - 0s 1ms/step - loss: 0.4460


In [8]:
#基于训练好的模型做出预测
X=X_batch[0]
pred = model.predict(X[np.newaxis,:, :])
print(pred.shape)

(1, 20, 1)
